In [1]:
import os
import time
import pandas as pd
import numpy as np

import rasterio

import dask_gateway
import dask.array as da

from joblib import load

import raster_to_features as rm
import data_sampling_workflow.sample_rasters as sr

import planetary_computer as pc
import rioxarray as rioxr
import xarray as xr
import dask.dataframe as daf
import dask.array as da

import math
from affine import Affine

In [2]:
n_workers = 16

In [3]:
# initialize DASK cluster
cluster = dask_gateway.GatewayCluster()
cluster.scale(n_workers)

client = cluster.get_client()
client

Connection method: Cluster object,Cluster type: dask_gateway.GatewayCluster
Dashboard: https://pccompute.westeurope.cloudapp.azure.com/compute/services/dask-gateway/clusters/prod.605ee14a357f4796ba17e3a38dcb677e/status,


In [52]:
actual_raster = rioxr.open_rasterio(href)
actual_raster

<xarray.DataArray (band: 4, y: 12500, x: 10580)>
[529000000 values with dtype=uint8]
Coordinates:
  * band         (band) int64 1 2 3 4
  * x            (x) float64 2.354e+05 2.354e+05 ... 2.418e+05 2.418e+05
  * y            (y) float64 3.815e+06 3.815e+06 ... 3.807e+06 3.807e+06
    spatial_ref  int64 0
Attributes:
    _FillValue:    0.0
    scale_factor:  1.0
    add_offset:    0.0

In [4]:
#scene_ids = pd.read_csv(os.path.join(os.getcwd(),'temp','coastal_scenes_ids_2020.csv')).itemid.loc[19:26]
itemid = 'ca_m_3411934_sw_11_060_20200521' # CAMPUS POINT 2020

# ---------------------------------------
# open NAIP scene
item = sr.get_item_from_id(itemid)   
y_n = item.properties['proj:shape'][0]
x_n = item.properties['proj:shape'][1]

href = pc.sign(item.assets["image"].href)

raster = rioxr.open_rasterio(href, chunks={"x": int(x_n/4), "y":int(y_n/4)})           # open raster as dask array (MAYBE FIGURE OUT BETTER CHUNK SIZES)

raster = raster.drop_vars(['spatial_ref','x','y'])
raster

<xarray.DataArray (band: 4, y: 12500, x: 10580)>
dask.array<open_rasterio-491a96a5fc7b7bf790408f36e12a7b61<this-array>, shape=(4, 12500, 10580), dtype=uint8, chunksize=(4, 3125, 2645), chunktype=numpy.ndarray>
Coordinates:
  * band     (band) int64 1 2 3 4
Dimensions without coordinates: y, x
Attributes:
    _FillValue:    0.0
    scale_factor:  1.0
    add_offset:    0.0

In [9]:
pixels = daf.from_array(raster.stack(z=("x", "y")).drop_vars('z').T, 
                      chunksize = int(x_n*y_n/n_workers))
pixels = pixels.astype('int16')
#pixels['index'] = pd.Series(np.arange(0,x_n*y_n))
pixels

,0,1,2,3
npartitions=16,,,,
0,int16,int16,int16,int16
8265625,...,...,...,...
...,...,...,...,...
123984375,...,...,...,...
132249999,...,...,...,...


In [10]:
pixels['ndvi'] = (pixels[3] - pixels[0])/(pixels[3] + pixels[0])
pixels['ndwi'] = (pixels[1] - pixels[3])/(pixels[1] + pixels[3])
pixels

,0,1,2,3,ndvi,ndwi
npartitions=16,,,,,,
0,int16,int16,int16,int16,float64,float64
8265625,...,...,...,...,...,...
...,...,...,...,...,...,...
123984375,...,...,...,...,...,...
132249999,...,...,...,...,...,...


In [11]:
pixels.compute()

,0,1,2,3,ndvi,ndwi
0,204,196,189,175,-0.076517,0.056604
1,204,196,188,175,-0.076517,0.056604
2,204,196,189,174,-0.079365,0.059459
3,177,181,169,152,-0.075988,0.087087
4,166,170,164,137,-0.095710,0.107492
...,...,...,...,...,...,...
8265620,29,48,61,13,-0.380952,0.573770
8265621,25,46,56,10,-0.428571,0.642857
8265622,23,45,56,10,-0.393939,0.636364
8265623,22,43,55,8,-0.466667,0.686275


In [12]:
not_water = pixels[pixels.ndwi < 0.3]
is_veg = not_water[not_water.ndvi > 0.05]

In [13]:
is_veg.compute()

,0,1,2,3,ndvi,ndwi
38,98,104,91,135,0.158798,-0.129707
39,111,115,102,133,0.090164,-0.072581
119,103,129,91,172,0.250909,-0.142857
120,71,108,72,178,0.429719,-0.244755
121,69,99,71,179,0.443548,-0.287770
...,...,...,...,...,...,...
8261730,22,43,55,25,0.063830,0.264706
8262979,51,87,63,65,0.120690,0.144737
8263763,24,40,47,35,0.186441,0.066667
8263764,27,69,48,42,0.217391,0.243243


In [8]:
index = is_veg.index
is_veg = is_veg.drop(['ndwi','index'], axis=1)

date = item.datetime
kwargs = {'year' : date.year,
         'month' : date.month,
         'day_in_year' : sr.day_in_year(date.day, date.month, date.year)}

is_veg = is_veg.assign(**kwargs)
is_veg

,0,1,2,3,ndvi,year,month,day_in_year
npartitions=16,,,,,,,,
0,int16,int16,int16,int16,float64,int64,int64,int64
8265625,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...
123984375,...,...,...,...,...,...,...,...
132249999,...,...,...,...,...,...,...,...


In [9]:
# open pre-trained random forest classifier
rfc = load('spectral_rfc.joblib') 
print('loaded model')

loaded model


In [10]:
da_pixels = is_veg.to_dask_array()
da_pixels

dask.array<values, shape=(nan, 8), dtype=float64, chunksize=(nan, 8), chunktype=numpy.ndarray>

In [16]:
samples = da_pixels.compute()
samples

array([[  98.,  104.,   91., ..., 2020.,    5.,  142.],
       [ 111.,  115.,  102., ..., 2020.,    5.,  142.],
       [ 103.,  129.,   91., ..., 2020.,    5.,  142.],
       ...,
       [  91.,  109.,   80., ..., 2020.,    5.,  142.],
       [  74.,   93.,   76., ..., 2020.,    5.,  142.],
       [  67.,   84.,   72., ..., 2020.,    5.,  142.]])

In [17]:
scene_preds = rfc.predict(samples)
scene_preds

array([0, 0, 0, ..., 0, 0, 0])

In [18]:
veg_index = index.compute()

In [19]:
veg_index

Int64Index([     38,      39,     119,     120,     121,     122,     123,
                124,     125,     126,
            ...
            8265583, 8265584, 8265585, 8265586, 8265594, 8265620, 8265621,
            8265622, 8265623, 8265624],
           dtype='int64', length=1755248)

In [20]:
preds_df = pd.DataFrame(scene_preds, 
                         columns=['is_iceplant'], 
                         index = veg_index)
is_iceplant_index = preds_df[preds_df.is_iceplant == 1].index.to_numpy()
non_iceplant_index = preds_df[preds_df.is_iceplant == 0].index.to_numpy()

In [24]:
# reconstruct indices into image
indices = [non_iceplant_index,
           is_iceplant_index]
values = [0,    # values assigned to pixels from each index
          1]
t0 = time.time()
reconstruct = rm.indices_to_image(x_n, y_n, indices, values, back_value=5)
print('reconstructed image\n   time taken to reconstruct: ', time.time() - t0,' s')

reconstructed image
   time taken to reconstruct:  0.510101318359375  s


In [29]:
reconstruct.shape

(10580, 12500)

In [48]:
# ---------------------------------------
# save raster 
filename = 'dask_preds_' + itemid +'.tif'

with rasterio.open(
    os.path.join(os.getcwd(),'temp',filename),  # file path
    'w',           # w = write
    driver = 'GTiff', # format
    height = y_n, 
    width = x_n,
    count = 1,  # number of raster bands in the dataset
    dtype = rasterio.uint8,
    crs = raster.rio.crs,
#    transform = raster.rio.transform(),
    transform = Affine(*item.properties['proj:transform'][0:6]),
) as dst:
    dst.write(reconstruct.astype(rasterio.uint8), 1)
# ---------------------------------------
print('FINISHED: ', itemid , '\n')

FINISHED:  ca_m_3411934_sw_11_060_20200521 



In [15]:
t0 = time.time()
preds = scene_preds.compute()
print('finished predicting\n   time taken to predict: ', time.time() - t0,' s')

ValueError: Found array with 0 sample(s) (shape=(0, 8)) while a minimum of 1 is required.